## 解决并发请求使代码重复执行

业务逻辑:

- 调用接口获取 `foo`, `bar` 数据
- 检查是否存在可用更新，如果存在则更新本地文件
- 读取本地 `manifest.json` 文件返回数据

存在问题:

- 存在重复执行更新本地文件
- 会导致读取了正在写入的文件时程序报错

In [1]:
var tasks = new Dictionary<string, Task>();

async Task Visit(string id)
{
    Task task;

    lock (tasks)
    {
        if (!tasks.TryGetValue(id, out task))
        {
            task = Task.Run(async () =>
            {
                Console.WriteLine("check for new version...");
                Console.WriteLine("new update available, writing file...");
                await Task.Delay(3000);
                Console.WriteLine("update success!");
            });

            tasks.Add(id, task);
        }
    }

    try
    {
        await task;
    }
    finally
    {
        tasks.Remove(id);
    }
}

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
// api/foo
async Task GetFoo(string id)
{
    await Visit(id);
    Console.WriteLine("read foo in manifest.json");

    // return foo
}

// api/bar
async Task GetBar(string id)
{
    await Visit(id);
    Console.WriteLine("read bar in manifest.json");

    // return bar
}

In [3]:
var task1 = GetFoo("1");
var task2 = GetBar("1");

Task.WaitAll(task1, task2);

check for new version...
new update available, writing file...
update success!
read foo in manifest.json
read bar in manifest.json


从上面的输出可以看出来， `Visit` 函数中的代码只被执行了一次